In [1]:
import os

ida_path = '/home/nahid/reverse/codes/ML/compare/ida/'
recursive_path = "/home/nahid/reverse/codes/ML/compare/recursive"


ida_files = os.listdir(ida_path)
recursive_files  = os.listdir(recursive_path)

In [15]:
import os,sys
from collections import defaultdict
import posixpath
from elftools.elf.elffile import ELFFile
from capstone import *




#look at dwarf lineprogram and get the concrete addresees
def get_lineprogram_addresses(elffile ,bin_bytearray):
    lineprogram_addresses = []
    
    text_section = elffile.get_section_by_name(".text")
    textSectionLen = len(bin_bytearray)
    textSectionStart = text_section.header.sh_addr
    textSectionEnd = textSectionStart + textSectionLen
    program_entry_point = elffile._parse_elf_header().e_entry




    if not elffile.has_dwarf_info():
        print('  file has no DWARF info')
    dwarfinfo = elffile.get_dwarf_info()
    cc = 0
    for CU in dwarfinfo.iter_CUs():


        # Every compilation unit in the DWARF information may or may not
        # have a corresponding line program in .debug_line.
        line_program = dwarfinfo.line_program_for_CU(CU)
        if line_program is None:
            print('  DWARF info is missing a line program for this CU')
            continue
        for le in line_program.get_entries():
            if le.state is not None:
                lineprogram_addresses.append(le.state.address)
    return lineprogram_addresses

In [19]:
for ida_fname in ida_files:
    if os.path.isfile(os.path.join(recursive_path,ida_fname)) == False:
        continue
    
    
    idaFilePath = os.path.join(ida_path,ida_fname)
    recurFilePath = os.path.join(recursive_path,ida_fname)
    
    
    ida_f = open(idaFilePath, 'r')
    recur_f = open(recurFilePath, 'r')
    
    ida_lines = ida_f.readlines()
    recur_lines = recur_f.readlines()
    
    ida_addresses = []
    for ida_l in ida_lines:
        ida_add = ida_l.split(':\t')[0]
        ida_add = int('0x'+ida_add, 16)
        ida_addresses.append(int(ida_add))
        
    
    
    
    recur_addresses = []
    for rec_l in recur_lines:
        rec_add = (rec_l.split(':\t')[0])

        rec_add = int(rec_add)
        recur_addresses.append(int(rec_add))
        
        
    ida_addresses = list(set(ida_addresses))
    recur_addresses = list(set(recur_addresses))
    print(len(ida_addresses), len(recur_addresses))
    
    
    exePath = '/home/nahid/reverse/codes/ML/data/binaries/'
    
    lineprogram_addresses = None
    with open(os.path.join(exePath,ida_fname.split('.txt')[0]), 'rb') as f:
        
        bin_bytearray = bytearray(f.read())
        elffile = ELFFile(f)

        lineprogram_addresses = get_lineprogram_addresses(elffile ,bin_bytearray)
        
    notin = 0 
    for lpa in lineprogram_addresses:
        if lpa not in ida_addresses:
            notin+=1
    print("not in ", notin)
    print("\n\n")
        

321 197
not in  0



34109 33137
not in  24



175 102
not in  1



295 165
not in  1



7992 7427
not in  9



